In [1]:
import os
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time

In [2]:
DATA_PATH = Path.cwd().parent / "data" / "raw"

## Display html of one page to see where to get the info

need to make https://www.veganrecipeclub.org.uk/recipes/15-minute-flourless-choc-chip-banana-muffins/ work

In [68]:
page = requests.get(recipes_urls[4])
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en-GB">
 <head>
  <meta charset="utf-8"/>
  <title>
   15 Minute Flourless Choc Chip Banana Muffins - Vegan Recipe Club
  </title>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="I don't usually toot my own horn, but these are genius! My daughter and husband both love them and I love making them for them. These muffins are not only..." name="description"/>
  <script type="text/javascript">
   function theChampLoadEvent(e){var t=window.onload;if(typeof window.onload!="function"){window.onload=e}else{window.onload=function(){t();e()}}}
  </script>
  <script type="text/javascript">
   var theChampDefaultLang = 'en_GB', theChampCloseIconPath = 'https://www.veganrecipeclub.org.uk/wp-content/plugins/super-socialize

In [69]:
#Title
soup.find_all("h1")[0].get_text(strip=True)

'15 Minute Flourless Choc Chip Banana Muffins'

In [77]:
soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

[<p style="text-align: center; font-weight: bold;">Preparation time</p>,
 <p style="text-align: center; font-weight: bold;">Cook time</p>,
 <p style="text-align: center; font-weight: bold;">Meal</p>,
 <p style="text-align: center; font-weight: bold;">Servings</p>,
 <p style="text-align: center; font-weight: bold;">Ingredients</p>,
 <p style="text-align: center; font-weight: bold;">Instructions</p>,
 <p style="text-align: center; font-weight: bold;">Keywords</p>]

In [70]:
#Preparation time
print(soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[0].nextSibling.nextSibling)
#Cook time
print(soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[1].nextSibling.nextSibling)
#Total time
print(soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[2].nextSibling.nextSibling)

<p style="text-align: center;"> </p>
<p style="text-align: center;"> </p>
<ul class="mealList">
<li>vegan-desserts</li>
<li>breakfasts</li>
</ul>


In [71]:
#Style of cooking
styles = soup.find_all("p", {"class": "intList"})
if len(styles) != 0:
    for style in styles:
        print(style.get_text(strip=True))

In [72]:
#Type of recipe
types = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[3].nextSibling.nextSibling.find_all("li")
for type_ in types:
    print(type_.get_text())

In [73]:
#Number of servings
soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[4].nextSibling.nextSibling.get_text()

'\n\n240g/1 cup tinned chickpeas, drained and rinsed\n60g/⅓ cup brown or coconut sugar\n3 tbsp maple syrup\n3 tbsp coconut oil, melted\n60g/¼ cup runny peanut butter (eg brands like Manilife and Pip & Nut are runnier than some of the thicker varieties on offer)\n60g/½ cup cacao powder\n1 tsp vanilla paste\n160ml/⅔ cup plant milk\n½ tsp bicarbonate of soda (baking soda)\n1 teaspoon baking powder\n\nTopping\n\n100g/3½ oz dark or milk dairy-free chocolate, chopped\n1 banana, sliced\n\n\n\n\n\n'

In [74]:
#Ingredients
HTML_ingredients = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[5].nextSibling.nextSibling
for item in HTML_ingredients.find_all("li"):
    print(item.get_text())
    
if(len(HTML_ingredients.find_all("li")) == 0):
    #Ingredients
    HTML_ingredients = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[5].parent.find("div", {"class": "recipe-list"}).p
    for ingredient in HTML_ingredients.get_text().split("\n"):
        print(ingredient)

Preheat the oven to 180°C (fan)/350°F/Gas Mark 4.
Line a muffin tin with 8 paper cases.
Put all of the ingredients into a blender and blitz until you have a beautiful smooth mixture.
Scoop the mixture into the 8 muffin cases. Put one or two pieces of chocolate into each muffin along with a slice or two of banana. Bake in the oven for 12–15 minutes. The muffins will be lovely and cooked on the outside but still gooey on the inside.
Leave them to cool on a wire rack, then enjoy.


In [75]:
#Instructions
HTML_instructions = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})[6].nextSibling.nextSibling
for item in HTML_instructions.find_all("li"):
    print(item.get_text())

Budget recipes
Family recipes
Guest chef
Recipes
Students
Easy
Desserts
Breakfasts
Gluten-free/optional
Wheat-free/optional
Soya-free/optional
Nut-free/optional
Kids
Teenagers
Plastic-free adaptable
Party food
Budget
Freezable
Father's Day
Mother's Day
Valentine's Day


## Scrape every urls

In [76]:
recipes_urls[:5]

['https://www.veganrecipeclub.org.uk/recipes/vegan-cherry-meringue-pie/',
 'https://www.veganrecipeclub.org.uk/recipes/smoked-tofu-rashers-grilled-tomatoes/',
 'https://www.veganrecipeclub.org.uk/recipes/patatas-bravas/',
 'https://www.veganrecipeclub.org.uk/recipes/mediterranean-wild-mushroom-pie/',
 'https://www.veganrecipeclub.org.uk/recipes/15-minute-flourless-choc-chip-banana-muffins/']